In [61]:
import pandas as pd
import os
import json
import subprocess
import re

In [9]:
# df = pd.DataFrame(columns=["Integration"])
def is_real_integration(name: str) -> bool:
    return os.path.isfile(f"homeassistant/components/{name}/__init__.py")
integrations = [x for x in os.listdir("homeassistant/components") if is_real_integration(x)]
integrations

['nest',
 'linux_battery',
 'decora_wifi',
 'spaceapi',
 'ffmpeg',
 'beewi_smartclim',
 'tensorflow',
 'oasa_telematics',
 'prosegur',
 'recorder',
 'modem_callerid',
 'growatt_server',
 'lirc',
 'circuit',
 'dormakaba_dkey',
 'kraken',
 'matrix',
 'scene',
 'remember_the_milk',
 'aquostv',
 'flo',
 'clickatell',
 'abode',
 'humidifier',
 'forecast_solar',
 'bizkaibus',
 'color_extractor',
 'notion',
 'horizon',
 'config',
 'netdata',
 'heos',
 'monoprice',
 'ddwrt',
 'fritzbox',
 'incomfort',
 'hydrawise',
 'otbr',
 'usgs_earthquakes_feed',
 'sma',
 'wsdot',
 'canary',
 'modbus',
 'ihc',
 'uvc',
 'familyhub',
 'ultraloq',
 'mitemp_bt',
 'openai_conversation',
 'rest_command',
 'network',
 'airthings',
 'todoist',
 'geonetnz_quakes',
 'blebox',
 'mold_indicator',
 'plex',
 'geonetnz_volcano',
 'energy',
 'plum_lightpad',
 'steamist',
 'weather',
 'nws',
 'wled',
 'efergy',
 'http',
 'baf',
 'gogogate2',
 'thinkingcleaner',
 'tilt_ble',
 'picnic',
 'lacrosse',
 'nuheat',
 'input_boolean

In [ ]:
df = pd.DataFrame(index=integrations, columns=["quality_scale", "config_flow", "sensor", "iot_class", "integration_type", "external_library", "codeowners", "has_tests", "coverage"])
def has_feature(name: str, featur: str) -> bool:
    return os.path.isfile(f"homeassistant/components/{name}/{featur}.py")
df = df.sort_index()
for integration_name, integration in df.iterrows():
    for feature in ("config_flow", "sensor"):
        df.at[integration_name, feature] = has_feature(integration_name, feature)
    with open(f"homeassistant/components/{integration_name}/manifest.json") as manifest_file:
        manifest = json.load(manifest_file)
    if "integration_type" in manifest:
        df.at[integration_name, "integration_type"] = manifest["integration_type"]
    if "iot_class" in manifest:
        df.at[integration_name, "iot_class"] = manifest["iot_class"]
    if "quality_scale" in manifest:
        df.at[integration_name, "quality_scale"] = manifest["quality_scale"]
    df.at[integration_name, "external_library"] = False
    if "requirements" in manifest:
        df.at[integration_name, "external_library"] = True
    df.at[integration_name, "codeowners"] = 0
    if "codeowners" in manifest:
        df.at[integration_name, "codeowners"] = len(manifest["codeowners"])
    has_tests = os.path.exists(f"tests/components/{integration_name}")
    df.at[integration_name, "has_tests"] = has_tests
    if has_tests:
        try:
            p = subprocess.Popen(["pytest", f"tests/components/{integration_name}", f"--cov=homeassistant.components.{integration_name}"], stdout=subprocess.PIPE)
            test, err = p.communicate()
            regex = r"TOTAL.*\s(\d*)%"
            result = re.search(regex, str(test))
            df.at[integration_name, "coverage"] = result.group(1)
        except:
            pass

df.head()

/home/joostlek/core/venv/lib/python3.11/site-packages/coverage/inorout.py:507: CoverageWarning: Module homeassistant.components.acmeda was never imported. (module-not-imported)
  self.warn(f"Module {pkg} was never imported.", slug="module-not-imported")
/home/joostlek/core/venv/lib/python3.11/site-packages/coverage/control.py:858: CoverageWarning: No data was collected. (no-data-collected)
  self._warn("No data was collected.", slug="no-data-collected")
/home/joostlek/core/venv/lib/python3.11/site-packages/pytest_cov/plugin.py:308: CovReportWarning: Failed to generate report: No data to report.

  warnings.warn(CovReportWarning(message))
/home/joostlek/core/venv/lib/python3.11/site-packages/coverage/inorout.py:507: CoverageWarning: Module homeassistant.components.adax was never imported. (module-not-imported)
  self.warn(f"Module {pkg} was never imported.", slug="module-not-imported")
/home/joostlek/core/venv/lib/python3.11/site-packages/coverage/control.py:858: CoverageWarning: No dat

In [49]:
df.to_excel("./state.xlsx")